# **Introducción a Scikit-learn**
------

### Selección automática de meta-parámetros

Habitualmente no se sabe de antemano la configuración óptima de los meta-parámetros (también llamados hiper-parámetros) de un modelo. Por ejemplo, en el modelo de k-vecinos visto anteriormente, no se sabe cuál es el valor óptimo de k.
Sklearn ofrece algunas utilidades para automatizar mediante fuerza bruta este proceso:

* GridSearchCV: permite definir explícitamente la rejilla en el espacio de meta-parámetros que se quiere explorar.
* RandomizedSearchCV: permite asignar distribuciones de probabilidad a los distintos meta-parámetros a explorar y prueba un número acotado de configuraciones posibles definidas por estas distribuciones.

GridSearchCV suele ser buena opción cuando hay pocos meta-parámetros a explorar y no tienen muchos valores posibles. Si el espacio de búsqueda es grande, sin embargo, su coste es excesivo y se prefiere RandomizedSearchCV, que tiene un coste acotado.

Existen alternativas un poco mejores que la simple fuerza bruta (ver BayesSearchCV de scikit-optimize), que generalmente son mejoras sobre RandomizedSearchCV en las que en lugar de buscar aleatoriamente se exploran regiones interesantes del espacio de meta-parámetros, pero en cualquier caso el proceso pasa por evaluar una gran cantidad de posibles configuraciones del modelo, multiplicando el coste con respecto al entrenamiento de una única configuración.

In [1]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

In [2]:
from sklearn.neighbors import KNeighborsClassifier
predictor = KNeighborsClassifier()

In [3]:
from sklearn.model_selection import GridSearchCV

In [4]:
search_space = {"n_neighbors": [1, 3, 5, 7, 9, 11]}
estimator = GridSearchCV(predictor, search_space)

In [5]:
estimator.fit(X, y)
print(f"Best hyper-params: {estimator.best_params_}\nBest accuracy: {estimator.best_score_}")

Best hyper-params: {'n_neighbors': 7}
Best accuracy: 0.9800000000000001


In [6]:
from sklearn.ensemble import RandomForestClassifier
predictor = RandomForestClassifier()

In [7]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [8]:
search_space = {"n_estimators": randint(1, 100),
                "criterion": ["gini", "entropy"],
                "max_depth": [2, 4, 8]}
estimator = RandomizedSearchCV(predictor, search_space, n_iter=10)

In [9]:
estimator.fit(X, y)
print(f"Best hyper-params: {estimator.best_params_}\nBest accuracy: {estimator.best_score_}")

Best hyper-params: {'criterion': 'entropy', 'max_depth': 4, 'n_estimators': 96}
Best accuracy: 0.9666666666666668


### Construcción de un modelo compatible con sklearn

En ocasiones el modelo buscado no está disponible en sklearn y no queda más opción que programarlo. En estos casos, es recomendable hacerlo compatible con sklearn para aprovechar todas las utilidades adicionales que la librería ofrece, como búsquedas de meta-parámetros, pipelines, cross-validators, métricas, etc.

Toda clase compatible con sklearn debe heredar de la clase BaseEstimator, y, en función de su objetivo, de al menos una de las clases siguientes:
* RegressorMixin
* ClassifierMixin
* ClusterMixin
* TransformerMixin

Además, debe implementar los métodos __init__, fit, score y predict (RegressorMixin, ClassifierMixin y ClusterMixin) o transform (TransformerMixin).

In [10]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import r2_score

class OLSRegressor(BaseEstimator, RegressorMixin):

    # Este modelo no tiene meta-parámetros, así que no requiere modificar el __init__ heredado de BaseEstimator
    
    def __init__(self):
        pass

    def fit(self, X, y):
        if len(X.shape) == 1: X = x.reshape((-1, 1))
        ones = np.ones((len(X), 1))
        X_bias = np.append(ones, X, axis=1)
        self.beta_ = X_inv @ X2 # Aquí se calcula Beta a partir de X_bias e y
        return self

    def predict(self, X):
        if len(X.shape) == 1: X = x.reshape((-1, 1))
        ones = np.ones((len(X), 1))
        X_bias = np.append(ones, X, axis=1)
        preds = X @ self.beta_ # Aquí se calculan las predicciones a partir de X_bias y self.beta_
        return preds
    
    def score(self,X,y):
        preds = self.predict(X)
        return r2_score(y,preds)

    # Este modelo no requiere una definición de score distinta de la r2 heredada de RegressorMixin

In [11]:
import numpy as np

In [12]:
X1 = X.T @ X

In [13]:
X_inv= np.linalg.inv(X1)
X_inv

array([[ 0.05034014, -0.0564344 , -0.04649057,  0.04508845],
       [-0.0564344 ,  0.06736108,  0.04506042, -0.03745986],
       [-0.04649057,  0.04506042,  0.06696879, -0.09831163],
       [ 0.04508845, -0.03745986, -0.09831163,  0.18358105]])

In [14]:
X2 = (X.T @ y)
X2

array([955.6, 435.9, 768.2, 268.9])

In [15]:
X_inv @ X2

array([-0.0844926 , -0.02356211,  0.22487123,  0.59972247])